## Colab com o código para realizar a segmentação das imagens do conjunto de teste utilizando o SAM 2 através do Ultralytics

In [ ]:
#Instala o pacote ultralytics incluindo todos os requisitos em um ambiente Python>=3.8 com PyTorch>=1.8 .
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 14.6 MB/s eta 0:00:00


In [ ]:
#Se carregar no colab um arquivo zip com o dataset, é necessário rodar o código abaixo para unzipar o conteúdo dessa pasta
#Unzipa o conteúdo da pasta zipada e salva os arquivos na pasta dataset
!unzip /content/plusme3.zip -d dataset/

Archive:  /content/plusme3.zip
   creating: dataset/plusme3/
   creating: dataset/plusme3/images/
   creating: dataset/plusme3/images/test/
  inflating: dataset/plusme3/images/test/asd7-103.jpg  
  inflating: dataset/plusme3/images/test/asd7-1044.jpg  
  inflating: dataset/plusme3/images/test/asd7-108.jpg  
  inflating: dataset/plusme3/images/test/asd7-1082.jpg  
  inflating: dataset/plusme3/images/test/asd7-1089.jpg  
  inflating: dataset/plusme3/images/test/asd7-1094.jpg  
  inflating: dataset/plusme3/images/test/asd7-1095.jpg  
  inflating: dataset/plusme3/images/test/asd7-1099.jpg  
  inflating: dataset/plusme3/images/test/asd7-1116.jpg  
  inflating: dataset/plusme3/images/test/asd7-1121.jpg  
  inflating: dataset/plusme3/images/test/asd7-1126.jpg  
  inflating: dataset/plusme3/images/test/asd7-1141.jpg  
  inflating: dataset/plusme3/images/test/asd7-1143.jpg  
  inflating: dataset/plusme3/images/test/asd7-1149.jpg  
  inflating: dataset/plusme3/images/test/asd7-1151.jpg  
  infla

In [ ]:
#Importações necessárias
#Importação do SAM através do pacote do ultralytics
#Importação do pandas
from ultralytics import SAM
import os
import pandas as pd

#Carrega o modelo do SAM2
model = SAM('sam2.1_b.pt')

#Define o caminho das imagens do conjunto de teste
imagens_teste = '/content/dataset/plusme3/images/test'
#Define o caminho dos labels das imagens do conjunto de teste
labels_imagens_teste = '/content/dataset/plusme3/labels/test'

#Define os nomes das colunas para o dataframe
#Os nomes foram definidos a partir dos dados que possui nos arquivos de labels das anotações das caixas delimitadoras
colunas = ["classe", "x", "y", "w", "h", "conf"]
largura = 1920  # Defina a largura da imagem
altura = 1080  # Defina a altura da imagem

# Lista de caminhos para imagens válidas (jpg e png)
caminhos_imagens = [os.path.join(imagens_teste, img) for img in os.listdir(imagens_teste)]

#Iteração para percorrer todos os arquivos de imagens
for caminho_imagem in caminhos_imagens:

    #Salva o caminho da imagem atual
    nome_arquivo = os.path.basename(caminho_imagem)
    #Salva o caminho do label da imagem
    #Dessa forma tanto o caminho da imagem como o caminho do label da imagem, está se referenciando a mesma imagem
    caminho_label_box = os.path.join(labels_imagens_teste, os.path.splitext(nome_arquivo)[0] + '.txt')

    #Verifica se encontra o arquivo de label da imagem
    if not os.path.exists(caminho_label_box):
        print(f"Arquivo de rótulo não encontrado: {caminho_label_box}")
        continue

    #Lê o arquivo de label da imagem que contém as anotações das caixas através do pandas
    #Salva os dados em formato de dataframe dividido nas colunas que foram definidas anteriormente
    df = pd.read_csv(caminho_label_box, delimiter=' ', header=None, names=colunas)

    #Iteração para percorrer todas as linhas do dataframe
    for i in range(len(df)):
        #variável que recebe os dados de uma linha específica a cada iteração
        dados = df.iloc[i]
        # As classes percorrem a cada linha, para acessar os valores de cada coluna, é preciso colocar os índices nas classes:
        # dados.iloc[0] = coluna classe
        # dados.iloc[1] = coluna x
        # dados.iloc[2] = coluna y
        # dados.iloc[3] = coluna w
        # dados.iloc[4] = coluna h
        # dados.iloc[5] = coluna conf
        # Desnormalizando os valores de xywhn -> xywh
        xywh = (
            dados.iloc[1] * largura,
            dados.iloc[2] * altura,
            dados.iloc[3] * largura,
            dados.iloc[4] * altura,
        )
        # Convertendo os valores de xywh -> xyxy (xinicio, yinicio, xfim, yfim)
        #xywh[0] = x
        #xywh[1] = y
        #xywh[2] = w
        #xywh[3] = h

        x_inicio = xywh[0] - xywh[2] / 2
        y_inicio = xywh[1] - xywh[3] / 2
        x_fim = xywh[0] + xywh[2] / 2
        y_fim = xywh[1] + xywh[3] / 2


        #Como no arquivo de labels possui algumas classes e dados referentes as caixas, verifica se a classe é 2.
        #Caso a classe seja 2, que se refere a classe do plusme, será feito a segmentação baseada nessa caixa. Então só é necessário os labels referentes a caixa do plusme, pois só será realizada a segmentação do plusme
        if dados.iloc[0] == 2:
          # Realiza a predição da segmentação na imagem atual baseada nas coordenadas da caixa do plusme, então é segmentado o conteúdo que está dentro dessa caixa. Também é salvo as imagens e labels resultantes dessa predição
          resultsMasks = model(caminho_imagem, bboxes=[x_inicio, y_inicio, x_fim, y_fim], save=True, save_txt=True, save_conf=True)

          # Define o nome de saída do arquivo das máscaras com base no nome da imagem
          nome_imagem = os.path.splitext(os.path.basename(caminho_imagem))[0]
          caminho_saida = os.path.join('labels_masks', f"{nome_imagem}_masks.txt")

          # Salva os dados da segmentação em um arquivo de texto
          with open(caminho_saida, "w") as f:
            for result in resultsMasks:
                if hasattr(result, 'masks'):  # Verifica se as máscaras estão disponíveis
                    for mask in result.masks:
                        f.write(f"{mask}\n") #Escreve os dados da máscara no arquivo de texto
                else:
                    print(f"Não há segmentos para a imagem {nome_imagem}")
else:
    print(f"Arquivo não encontrado ou inválido: {caminho_imagem}")





image 1/1 /content/dataset/plusme3/images/test/asd7-103.jpg: 1024x1024 1 0, 16244.8ms
Speed: 18.7ms preprocess, 16244.8ms inference, 20.1ms postprocess per image at shape (1, 3, 1024, 1024)
Results saved to runs/segment/predict3
1 label saved to runs/segment/predict3/labels

image 1/1 /content/dataset/plusme3/images/test/asd7-50.jpg: 1024x1024 1 0, 15404.6ms
Speed: 11.1ms preprocess, 15404.6ms inference, 20.0ms postprocess per image at shape (1, 3, 1024, 1024)
Results saved to runs/segment/predict3
2 labels saved to runs/segment/predict3/labels



KeyboardInterrupt: 